In [1]:
# !pip install biopython 
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import pandas as pd
import statistics 

# For Alpha
file = "C:\\Users\\shree\\Desktop\\ACADS\\Comp Bio Lab\\alpha_40.txt"
Big_sequence = ''
df = pd.DataFrame()

sequences = SeqIO.parse(open(file),'fasta')
for file in sequences:
    sequence = str(file.seq)
    x = ProteinAnalysis(sequence)
    df_small = pd.DataFrame([dict(x.get_amino_acids_percent())])
    df = pd.concat([df, df_small], ignore_index=True)
    Big_sequence += sequence
X = ProteinAnalysis(Big_sequence)

df_Big_a = pd.DataFrame([dict(X.get_amino_acids_percent())])
df_alpha = pd.concat([df, df_Big_a], ignore_index=True)
df_individual_alpha = df_alpha[:-1]

# For Beta
file = "C:\\Users\\shree\\Desktop\\ACADS\\Comp Bio Lab\\beta_40.txt"
Big_sequence = ''
df = pd.DataFrame()

sequences = SeqIO.parse(open(file),'fasta')
for file in  sequences:
    sequence = str(file.seq)
    x = ProteinAnalysis(sequence)
    df_small = pd.DataFrame([dict(x.get_amino_acids_percent())])
    df = pd.concat([df, df_small], ignore_index=True)
    Big_sequence += sequence
X = ProteinAnalysis(Big_sequence)

df_Big_b = pd.DataFrame([dict(X.get_amino_acids_percent())])
df_beta = pd.concat([df, df_Big_b], ignore_index=True)
df_individual_beta = df_beta[:-1]

print('The overall amino acid percentage in TMH is')
print(round(df_Big_a,3))

print('\n The overall amino acid percentage in TMB is')
print(round(df_Big_b,3))

The overall amino acid percentage in TMH is
       A      C     D      E      F      G      H      I      K      L      M  \
0  0.084  0.015  0.04  0.048  0.056  0.069  0.019  0.068  0.045  0.118  0.027   

       N      P      Q      R      S      T      V      W      Y  
0  0.037  0.042  0.034  0.046  0.062  0.054  0.077  0.018  0.036  

 The overall amino acid percentage in TMB is
       A      C      D      E      F      G      H      I     K      L      M  \
0  0.076  0.007  0.064  0.048  0.042  0.089  0.015  0.045  0.05  0.081  0.017   

       N      P      Q     R      S      T      V      W      Y  
0  0.061  0.038  0.043  0.05  0.076  0.067  0.064  0.017  0.049  


In [2]:
# FDR Calculations
mean_alpha = pd.DataFrame([dict(df_individual_alpha.mean())])
std_alpha = pd.DataFrame([dict(df_individual_alpha.std())])
mean_beta = pd.DataFrame([dict(df_individual_beta.mean())])
std_beta = pd.DataFrame([dict(df_individual_beta.std())])

FDR = pd.concat([mean_alpha, mean_beta, std_alpha, std_beta], ignore_index=True)

FDR_list = []
for column in FDR:
    ma = FDR[column][0]
    mb = FDR[column][1]
    sa = FDR[column][2]
    sb = FDR[column][3]

    FDR_val = (ma-mb)**2/((sa**2)+(sb**2))
    FDR_list.append(round(FDR_val,3)) #Rounded off to 3rd decimal place

FDR.loc[4] = FDR_list
round(FDR,3)
FDR["Index"] = ["Mean alpha", "Mean Beta", "Std Alpha", "Std Beta", "FDR Value"]
FDR.insert(0,"Index", FDR.pop("Index"))

FDR = round(FDR,3)

print(FDR)

        Index      A      C      D      E      F      G      H      I      K  \
0  Mean alpha  0.086  0.015  0.040  0.049  0.055  0.071  0.019  0.068  0.049   
1   Mean Beta  0.081  0.006  0.062  0.049  0.043  0.088  0.016  0.044  0.049   
2   Std Alpha  0.035  0.019  0.021  0.028  0.025  0.029  0.014  0.030  0.029   
3    Std Beta  0.034  0.012  0.019  0.020  0.018  0.029  0.012  0.019  0.022   
4   FDR Value  0.014  0.150  0.633  0.000  0.155  0.183  0.041  0.434  0.000   

   ...      M      N      P      Q      R      S      T      V      W      Y  
0  ...  0.026  0.036  0.044  0.033  0.047  0.060  0.053  0.077  0.019  0.037  
1  ...  0.017  0.059  0.039  0.044  0.050  0.074  0.065  0.066  0.018  0.047  
2  ...  0.016  0.018  0.021  0.018  0.024  0.023  0.020  0.026  0.014  0.018  
3  ...  0.011  0.025  0.019  0.018  0.023  0.021  0.021  0.020  0.012  0.019  
4  ...  0.258  0.563  0.028  0.171  0.007  0.210  0.160  0.115  0.004  0.158  

[5 rows x 21 columns]


In [3]:
# TMH Calculations
sigmaTHM_alpha = []
sigmaTHB_alpha = []
for i in range(len(df_individual_alpha)):
    comp1 = 0
    comp2 = 0 
    for j in range(20):
        comp1 += abs(df_individual_alpha.iloc[i,j] - df_Big_a.iloc[0,j])
        comp2 += abs(df_individual_alpha.iloc[i,j] - df_Big_b.iloc[0,j])
        
    sigmaTHM_alpha.append(comp1)
    sigmaTHB_alpha.append(comp2)

TP = 0
FN = 0
delta = 0
for k in range(len(sigmaTHM_alpha)):
    if (sigmaTHM_alpha[k] + delta) < sigmaTHB_alpha[k]:
        TP += 1
    else:
        FN += 1

# TMB Calculations
sigmaTHB_beta = []
sigmaTHM_beta = []

for i in range(len(df_individual_beta)):
    comp1 = 0
    comp2 = 0 
    for j in range(20):
        comp1 += abs(df_individual_beta.iloc[i,j] - df_Big_b.iloc[0,j])
        comp2 += abs(df_individual_beta.iloc[i,j] - df_Big_a.iloc[0,j])
        
    sigmaTHB_beta.append(comp1)
    sigmaTHM_beta.append(comp2)

TN = 0
FP = 0
delta = 0
for k in range(len(sigmaTHB_beta)):
    if (sigmaTHB_beta[k] + delta) < sigmaTHM_beta[k]:
        TN += 1
    else:
        FP += 1

# Computations
Sensitivity = round(TP/(TP+FN),3)
Specificity = round(TN/(TN+FP),3)
Accuracy = round((TP+TN)/(TP+TN+FP+FN),3)

print(f'Sensitivity = {Sensitivity}')
print(f'Specificity = {Specificity}')
print(f'Accuracy = {Accuracy}')

Sensitivity = 0.803
Specificity = 0.835
Accuracy = 0.806


In [4]:
# percentage composition
def split(p):
    file = "C:\\Users\\shree\\Desktop\\ACADS\\Comp Bio Lab\\alpha_40.txt"
    Big_sequence = ''

    sequences = SeqIO.parse(open(file),'fasta')
    count = 0
    for file in sequences:
        if count == int(2353*(p/100)):
            break
        sequence = str(file.seq)
        Big_sequence += sequence
        count += 1
    X = ProteinAnalysis(Big_sequence)
    
    df_Big_a = pd.DataFrame([dict(X.get_amino_acids_percent())])

    # For Beta
    file = "C:\\Users\\shree\\Desktop\\ACADS\\Comp Bio Lab\\beta_40.txt"
    Big_sequence = ''
    
    sequences = SeqIO.parse(open(file),'fasta')
    count = 0
    for file in  sequences:
        if count == int(243*(p/100)):
            break            
        sequence = str(file.seq)
        Big_sequence += sequence
        count += 1
    X = ProteinAnalysis(Big_sequence)  
    df_Big_b = pd.DataFrame([dict(X.get_amino_acids_percent())])

    return df_Big_a, df_Big_b

In [5]:
# p%
p = 70
k1 = len(df_alpha)
k2 = len(df_beta)

df_alpha_1 = df_individual_alpha[int(k1*p/100):]
df_beta_1 = df_individual_beta[int(k2*p/100):]

df_Big_a, df_Big_b = split(p)


# TMH Calculations
sigmaTHM_alpha = []
sigmaTHB_alpha = []
for i in range(len(df_alpha_1)):
    comp1 = 0
    comp2 = 0 
    for j in range(20):
        comp1 += abs(df_alpha_1.iloc[i,j] - df_Big_a.iloc[0,j])
        comp2 += abs(df_alpha_1.iloc[i,j] - df_Big_b.iloc[0,j])
        
    sigmaTHM_alpha.append(comp1)
    sigmaTHB_alpha.append(comp2)

TP = 0
FN = 0
for k in range(len(sigmaTHM_alpha)):
    if sigmaTHM_alpha[k] < sigmaTHB_alpha[k]:
        TP += 1
    else:
        FN += 1

# TMB Calculations
sigmaTHB_beta = []
sigmaTHM_beta = []

for i in range(len(df_beta_1)):
    comp1 = 0
    comp2 = 0 
    for j in range(20):
        comp1 += abs(df_beta_1.iloc[i,j] - df_Big_b.iloc[0,j])
        comp2 += abs(df_beta_1.iloc[i,j] - df_Big_a.iloc[0,j])
        
    sigmaTHB_beta.append(comp1)
    sigmaTHM_beta.append(comp2)

TN = 0
FP = 0
for k in range(len(sigmaTHB_beta)):
    if sigmaTHB_beta[k] < sigmaTHM_beta[k]:
        TN += 1
    else:
        FP += 1

# Computations
Sensitivity = round(TP/(TP+FN),3)
Specificity = round(TN/(TN+FP),3)
Accuracy = round((TP+TN)/(TP+TN+FP+FN),3)

print(f'Split = {p} %')
print(f'Sensitivity = {Sensitivity}')
print(f'Specificity = {Specificity}')
print(f'Accuracy = {Accuracy}')

Split = 70 %
Sensitivity = 0.824
Specificity = 0.74
Accuracy = 0.816
